# PART I - Data Extraction (SQL)


## Shopper Recurrence Rate metric calculation

1. We are going to download de raw data from de GitHub repository:

In [3]:
 url = 'https://raw.githubusercontent.com/cpallasvadillo/sequra-analytics-challenge/refs/heads/part_1_sql/orders_merchant.csv?token=GHSAT0AAAAAADKHMXUECMZNYU2FKWQTBHOE2FWWRHQ'
 !wget -O orders_merchant.csv "{url}"

--2025-09-02 09:08:49--  https://raw.githubusercontent.com/cpallasvadillo/sequra-analytics-challenge/refs/heads/part_1_sql/orders_merchant.csv?token=GHSAT0AAAAAADKHMXUECMZNYU2FKWQTBHOE2FWWRHQ
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 746 [text/plain]
Saving to: ‘orders_merchant.csv’

orders_merchant.csv 100%[===================>]     746  --.-KB/s    in 0s      

2025-09-02 09:08:49 (37.4 MB/s) - ‘orders_merchant.csv’ saved [746/746]



2. We use DuckDB because it's the most directly and easy way to work with SQL for this exercise:

In [4]:
!pip install duckdb pandas

In [7]:
import duckdb
import pandas as pd

# Step 1: Save the file's name
csv_file_name = 'orders_merchant.csv'

# Step 2: Identify de two tables
print(f"Reading de two tables of the '{csv_file_name}' file ...")

# Step 2.1: Table 1 - Orders
print("\nReading 'orders' table ...")
ORDERS = pd.read_csv(csv_file_name, header = 0, nrows=25, sep=';')

# Step 2.2: Table 2 - Merchants
print("Reading 'merchants' table ...")
MERCHANTS = pd.read_csv(csv_file_name, header = 27, nrows=6, sep=';', usecols=range(2))

# Step 3: Table verification
print("\nOrders Table")
display(ORDERS)

print("\nMerchants Table")
display(MERCHANTS)

Reading de two tables of the 'orders_merchant.csv' file ...

Reading 'orders' table ...
Reading 'merchants' table ...

Orders Table


,order_id,shopper_id,merchant_id,order_date
0,1,1234,9876,9/9/22
1,2,1235,9875,10/9/22
2,3,1236,9877,11/9/22
3,4,1238,9876,13/10/22
4,5,1237,9876,12/10/22
5,6,1239,9876,14/10/22
6,7,1239,9876,15/10/22
7,8,1240,9875,16/11/22
8,9,1234,9875,17/11/22
9,10,1235,9875,18/11/22



Merchants Table


,merchant_id,merchant_name
0,9876,Merchant A
1,9875,Merchant B
2,9877,Merchant C
3,9885,Merchant D
4,9886,Merchant E
5,9887,Merchant F


In [10]:
# Execution of the SQL Query

# Step 1: Connect DuckDB
con = duckdb.connect(database=':memory:')

# Step 2: Query Definition
query = """
WITH DISTINCT_SHOPPERS AS (
    SELECT
        ORDERS.SHOPPER_ID,
        ORDERS.MERCHANT_ID,
        EXTRACT(YEAR FROM strptime(ORDERS.ORDER_DATE, '%d/%m/%y')) AS ANYO,
        EXTRACT(MONTH FROM strptime(ORDERS.ORDER_DATE, '%d/%m/%y')) AS MES,
        (EXTRACT(YEAR FROM strptime(ORDERS.ORDER_DATE, '%d/%m/%y')) * 100 + EXTRACT(MONTH FROM strptime(ORDERS.ORDER_DATE, '%d/%m/%y'))) AS ANYOMES
    FROM ORDERS
    GROUP BY
        ORDERS.SHOPPER_ID,
        ORDERS.MERCHANT_ID,
        EXTRACT(YEAR FROM strptime(ORDERS.ORDER_DATE, '%d/%m/%y')),
        EXTRACT(MONTH FROM strptime(ORDERS.ORDER_DATE, '%d/%m/%y'))
),

RECURRENT_SHOPPERS AS (
    SELECT
        CURRENT_MONTH.SHOPPER_ID,
        CURRENT_MONTH.MERCHANT_ID,
        CURRENT_MONTH.ANYOMES,
        CASE WHEN COUNT(PREVIOUS_MONTH.SHOPPER_ID) > 0 THEN 1 ELSE 0 END AS IND_RECURRENT
    FROM DISTINCT_SHOPPERS AS CURRENT_MONTH
    LEFT JOIN DISTINCT_SHOPPERS AS PREVIOUS_MONTH
    ON
        CURRENT_MONTH.SHOPPER_ID = PREVIOUS_MONTH.SHOPPER_ID
        AND PREVIOUS_MONTH.ANYOMES < CURRENT_MONTH.ANYOMES
        AND PREVIOUS_MONTH.ANYOMES >= ((CURRENT_MONTH.ANYO - 1) * 100 + CURRENT_MONTH.MES)
    GROUP BY
        CURRENT_MONTH.SHOPPER_ID,
        CURRENT_MONTH.MERCHANT_ID,
        CURRENT_MONTH.ANYOMES
)

SELECT
    MERCHANTS.MERCHANT_NAME,
    RECURRENT_SHOPPERS.ANYOMES,
    (SUM(RECURRENT_SHOPPERS.IND_RECURRENT) * 100.0 / COUNT(DISTINCT RECURRENT_SHOPPERS.SHOPPER_ID)) AS RECURRENCE_RATE

    FROM RECURRENT_SHOPPERS
	left JOIN MERCHANTS
	ON MERCHANTS.MERCHANT_ID = RECURRENT_SHOPPERS.MERCHANT_ID

    GROUP BY MERCHANTS.MERCHANT_NAME, RECURRENT_SHOPPERS.ANYOMES
	ORDER BY MERCHANTS.MERCHANT_NAME, RECURRENT_SHOPPERS.ANYOMES;
"""

# Step 3: Query execution
result = con.execute(query).fetchdf()

# Step 4: Print the result
display(result)

,merchant_name,ANYOMES,RECURRENCE_RATE
0,Merchant A,202209,0.000000
1,Merchant A,202210,33.333333
2,Merchant B,202209,0.000000
3,Merchant B,202211,100.000000
4,Merchant B,202212,100.000000
5,Merchant C,202209,0.000000
6,Merchant C,202212,100.000000
7,Merchant D,202209,0.000000
8,Merchant D,202212,100.000000
9,Merchant E,202210,100.000000
